<a href="https://colab.research.google.com/github/talaa/AI_CrewAI/blob/main/Crew_AI__test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install 'crewai[tools]'

In [4]:
from google.colab import userdata
OpenAPIKey=userdata.get('OpenAPIKEY')
SearchAPIKey=userdata.get('GoogleSearchAPI')

In [5]:
import os
os.environ["SERPER_API_KEY"] = SearchAPIKey # serper.dev API key
os.environ["OPENAI_API_KEY"] = OpenAPIKey

In [14]:
from crewai import Agent
from crewai_tools import SerperDevTool
from langchain_openai import OpenAI
search_tool = SerperDevTool()

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True,
  llm=OpenAI(
        temperature=0.8, model_name="gpt-3.5-turbo-instruct"
    ),  # It uses langchain.chat_models, default is GPT 3.5
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False,
  llm=OpenAI(
        temperature=0.8,model_name="gpt-3.5-turbo-instruct"
    ),  # It uses langchain.chat_models, default is GPT 3.5
)

In [15]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [16]:
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential  # Optional: Sequential task execution is default
)

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'The future using Agentic AI'})
print(result)